In [1]:
import pandas as pd
import os
import numpy as np
import requests
import datetime
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import datetime
import time
import json
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.ticker import MultipleLocator
import openmeteo_requests
import requests_cache
from retry_requests import retry
import hsfs
from pathlib import Path
from dotenv import load_dotenv
import hopsworks
import sys

root_dir = Path().resolve().parent
sys.path.append(str(root_dir))

from format_data import format_weather_data, format_price_data, process_weather_data
from get_electricity_prices import get_data
from get_weather_data import get_historical_weather, get_weather_forecast
from entsoe_data import fetch_historical_data, ensure_valid_series



In [2]:
import hsfs

In [3]:
import hopsworks

In [2]:
load_dotenv()
entose_api = os.getenv("ENTSOE_API")
hopsworks_api = os.getenv("HOPSWORKS_API_KEY")

os.environ["HOPSWORKS_API_KEY"] = hopsworks_api

project = hopsworks.login()
print(f"Connected to project: {project.name}")

2025-01-05 14:38:07,334 INFO: Initializing external client
2025-01-05 14:38:07,335 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-05 14:38:08,797 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164446
Connected to project: oskaralf


In [8]:
start_date = "2022-11-01"
end_date = "2025-01-05"
end_date = datetime.date.today().strftime('%Y-%m-%d')
hist_weather_df = get_historical_weather("Stockhom", "2022-11-01", "2025-01-05", 59.3294, 18.0687)
formatted_hist_weather_df = process_weather_data(hist_weather_df)

Coordinates 59.29701232910156°N 18.163265228271484°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [9]:
print(formatted_hist_weather_df.head())

                 date  temperature_2m  precipitation  snow_depth  \
0 2022-11-01 00:00:00        9.165000            0.0         0.0   
1 2022-11-01 01:00:00        8.764999            0.0         0.0   
2 2022-11-01 02:00:00        8.615000            0.0         0.0   
3 2022-11-01 03:00:00        8.565000            0.0         0.0   
4 2022-11-01 04:00:00        8.365000            0.0         0.0   

   pressure_msl  cloud_cover  wind_speed_10m  wind_speed_100m  \
0   1017.099976        100.0        9.114471        17.698677   
1   1017.200012        100.0        8.427383        16.622490   
2   1017.200012        100.0        8.427383        16.454008   
3   1017.099976        100.0        8.759178        17.283749   
4   1016.900024        100.0        8.640000        17.106628   

   wind_direction_10m  wind_direction_100m      city  
0           260.90979           274.666779  Stockhom  
1           250.01680           265.030334  Stockhom  
2           250.01680           259

In [10]:
entsoe_df = fetch_historical_data(entose_api, start_date, end_date)

[DEBUG] Successfully fetched cross-border flows SE3 to Finland.
[DEBUG] Successfully fetched cross-border flows Finland to SE3.
[DEBUG] Successfully fetched cross-border flows SE3 to Norway.
[DEBUG] Successfully fetched cross-border flows Norway to SE3.
[DEBUG] Successfully fetched cross-border flows SE3 to Denmark.
[DEBUG] Successfully fetched cross-border flows Denmark to SE3.


In [12]:
#print(formatted_hist_weather_df.head())
print(entsoe_df.head())

                           historical_load  historical_day_ahead_prices  \
index                                                                     
2022-10-31 23:00:00+00:00           7877.0                        34.91   
2022-11-01 00:00:00+00:00           7673.0                        34.91   
2022-11-01 01:00:00+00:00           7504.0                        35.31   
2022-11-01 02:00:00+00:00           7471.0                        36.11   
2022-11-01 03:00:00+00:00           7518.0                        37.83   

                           historical_total_generation_Fossil Gas  \
index                                                               
2022-10-31 23:00:00+00:00                                     0.0   
2022-11-01 00:00:00+00:00                                     0.0   
2022-11-01 01:00:00+00:00                                     0.0   
2022-11-01 02:00:00+00:00                                     0.0   
2022-11-01 03:00:00+00:00                                   

In [16]:
# price_df = get_data("SE3")
# print(price_df.dtypes)
# print(price_df)

In [15]:
# price_df['time_start'] = price_df['time_start'].str.replace(r'\+\d{2}:\d{2}$', '', regex=True)
# print(price_df)
# print(price_df.dtypes)

In [13]:
# formatted_hist_weather_df['time_start'] = formatted_hist_weather_df['time_start'].str.replace(r'\+\d{2}:\d{2}$', '', regex=True)
# print(formatted_hist_weather_df)
# print(formatted_hist_weather_df.dtypes)


In [14]:
# print(price_df)
# print(price_df.dtypes)

In [17]:

# print(price_df.head())
# print(price_df.dtypes)

In [11]:
import great_expectations as ge

weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"temperature_2m",
            "min_value":-100.0,
            "max_value":500.0,
        }
    )
)

weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"precipitation",
            "min_value":-5.0,
            "max_value":500.0,

        }
    )
)

weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"pressure_msl",
            "min_value":0.0,
            "max_value":2000,
        }
    )
)



weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"cloud_cover",
            "min_value":-0.1,
            "max_value":100.0,
            "strict_min":True
        }
    )
)

weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"wind_speed_10m",
            "min_value":-0.1,
            "max_value":1000,
            "strict_min":True
        }
    )
)



{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "wind_speed_10m", "min_value": -0.1, "max_value": 1000, "strict_min": true}, "meta": {}}

In [10]:
import great_expectations as ge

prices_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="price_expectation_suite"
)

prices_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"price",
            "min_value":-0.1,
            "max_value":5000,
            "strict_min":True
        }
    )
)

{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "price", "min_value": -0.1, "max_value": 5000, "strict_min": true}, "meta": {}}

In [12]:
import great_expectations as ge

data_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="data_expectation_suite"
)

data_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "load_se",
            "min_value": 0,
            "max_value": 20000,
            "strict_min": True,
        }
    )
)

data_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "price_se",
            "min_value": -1000,
            "max_value": 5000,
        }
    )
)

data_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "flows_se_finland",
            "min_value": -5000,
            "max_value": 10000,  
        }
    )
)

data_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "flows_se_norway",
            "min_value": -5000,
            "max_value": 10000,
        }
    )
)

data_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "flows_se_denmark",
            "min_value": -5000,
            "max_value": 10000,
        }
    )
)

data_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "load_finland",
            "min_value": 0,
            "max_value": 20000,
        }
    )
)

data_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "load_norway",
            "min_value": 0,
            "max_value": 20000,
        }
    )
)

data_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "load_germany",
            "min_value": 0,
            "max_value": 100000,
        }
    )
)

data_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "load_denmark",
            "min_value": 0,
            "max_value": 20000,
        }
    )
)

data_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "hydro_storage_se",
            "min_value": 0,
            "max_value": 4000000,
        }
    )
)

print("Expectation suite created!")


Expectation suite created!


In [13]:
try:
    fs = project.get_feature_store()
    print(f"Connected to feature store: {fs.name}")
except Exception as e:
    print(f"Failed to get feature store: {e}")

Connected to feature store: oskaralf_featurestore


In [14]:
weather_fg = fs.get_or_create_feature_group(
    name='weather_data_2',
    description='Weather data for SE3',
    version=1,
    primary_key=['date'],
    expectation_suite=weather_expectation_suite
)

In [15]:
entsoe_fg = fs.get_or_create_feature_group(
    name='entsoe_data',
    description='Entsoe data for SE3',
    version=1,
    primary_key=['date'],
    expectation_suite=data_expectation_suite
)

In [ ]:
price_fg = fs.get_or_create_feature_group(
    name='electricity_price_data_3',
    description='Electricity price data for SE3',
    version=1,
    primary_key=['time_start'],
    expectation_suite=prices_expectation_suite
)

In [16]:
weather_fg.insert(formatted_hist_weather_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1394547
2025-01-04 20:42:27,069 INFO: 	5 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1394547


Uploading Dataframe: 100.00% |██████████| Rows 18456/18456 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_data_2_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/weather_data_2_1_offline_fg_materialization/executions


(Job('weather_data_2_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pressure_msl",
           "min_value": 0.0,
           "max_value": 2000
         },
         "meta": {
           "expectationId": 694345
         }
       },
       "result": {
         "observed_value": 962.7999877929688,
         "element_count": 18456,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-04T07:42:27.000068Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_betw

In [17]:
entsoe_df.columns = entsoe_df.columns.str.lower()
entsoe_fg.insert(entsoe_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393532
2025-01-04 20:42:47,235 INFO: 	10 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393532


Uploading Dataframe: 100.00% |██████████| Rows 19057/19057 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: entsoe_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/entsoe_data_1_offline_fg_materialization/executions


(Job('entsoe_data_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "flows_se_finland",
           "min_value": -5000,
           "max_value": 10000
         },
         "meta": {
           "expectationId": 696021
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 19057,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-04T07:42:47.000234Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
    

In [22]:
price_fg.insert(price_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393146
2024-12-23 12:41:23,583 INFO: 	1 expectation(s) included in expectation_suite.
Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393146


Uploading Dataframe: 100.00% |██████████| Rows 18815/18815 | Elapsed Time: 00:06 | Remaining Time: 00:00


Launching job: electricity_price_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/electricity_price_data_1_offline_fg_materialization/executions


(Job('electricity_price_data_1_offline_fg_materialization', 'SPARK'),
 {
   "success": false,
   "results": [
     {
       "success": false,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "price",
           "min_value": -0.1,
           "max_value": 5000,
           "strict_min": true
         },
         "meta": {
           "expectationId": 695305
         }
       },
       "result": {
         "observed_value": -0.69112,
         "element_count": 18815,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-12-23T11:41:23.000582Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
    